Import packages and libraries

In [1]:
import time
from pynq import Overlay
import matplotlib.pyplot as plt
import math

start_time = time.time()

#import IP
ol = Overlay("design_1_wrapper.bit")

#start the timer
start_time = time.time()

ModuleNotFoundError: No module named 'pynq'

ol?

Running the same script as Koch-Snowflake-CPU except this time all calculations are done using custom IP core.

In [ ]:
koch_ip = ol.koch_snowflake_0

# Function to draw a Koch curve with a given order
def koch_lines(x, y, a, angle, order):
    if order == 0:
        # Draw a straight line
        # Calculate values on IP core

        # x
        koch_ip.write(0x04, int(x))
        koch_ip.write(0x10, int(a))
        koch_ip.write(0x08, int(angle))

        x_end = koch_ip.read(0x14)  # read x_end

        # y
        koch_ip.write(0x00, int(y))
        koch_ip.write(0x10, int(a))
        koch_ip.write(0x08, int(angle))

        y_end = koch_ip.read(0x1C)  # read y_end

        # cpu math calculation
        #x_end = x + a * math.cos(math.radians(angle))
        #y_end = y + a * math.sin(math.radians(angle))

        plt.plot([x, x_end], [y, y_end], color='blue')

    else:
        # Recursive calls for each segment of the Koch curve
        #a /= 3  # Divide segment length by 3 for each iteration
        
        #for angle_offset in [0, 60, -60, 0]:
        for angle_offset in [0, 1.0472, -1.0472]:
            koch_lines(x, y, a, angle + angle_offset, order - 1)
            #x = x + a * math.cos(math.radians(angle + angle_offset))
            #y = y + a * math.sin(math.radians(angle + angle_offset))

            #x
            koch_ip.write(0x04, int(x))
            koch_ip.write(0x10, int(a)) #a/3
            koch_ip.write(0x08, int(angle))
            koch_ip.write(0x0C, int(angle_offset))

            x = koch_ip.read(0x18)

            #y
            koch_ip.write(0x00, int(y))
            koch_ip.write(0x10, int(a)) #a/3
            koch_ip.write(0x08, int(angle))
            koch_ip.write(0x0C, int(angle_offset))

            y = koch_ip.read(0x20)



def koch_snowflake(x, y, a, order):
    # Plot three Koch curves to form a triangle
    #for angle in [0, -120, 120]:
    for angle in [0, -2.0944, 2.0944]:
        #print(angle)
        koch_lines(x, y, a, angle, order)
        #x = x + a * math.cos(math.radians(angle))
        #x
        koch_ip.write(0x04, int(x))
        koch_ip.write(0x10, int(a)) #a/3
        koch_ip.write(0x08, int(angle))

        x = koch_ip.read(0x18)
        #y = y + a * math.sin(math.radians(angle))
        #y
        koch_ip.write(0x00, int(y))
        koch_ip.write(0x10, int(a)) #a/3
        koch_ip.write(0x08, int(angle))

        y = koch_ip.read(0x20)


Print the image to the screen and track the time taken

In [ ]:
# Create a figure
fig = plt.figure("Koch Snowflake")

# Plot the Koch snowflake
n = 1  # order of the Koch snowflake    -- anything above 7/8 you begin to run out of memory
koch_snowflake(0, 0, 1, n)  # Adjust the parameters as needed

# time to execute
end_time = time.time()
print("Time taken: ", end_time - start_time, "seconds")

# Show the plot
plt.axis('equal')
plt.axis('off')
plt.show()
